# MRI NIH : Data loading

## Imports

In [5]:
import os
import torch
import numpy as np
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils
from msct_image import Image

In [ ]:
class SegmentationSet2D(object):
    """This class is used to build 2D segmentation datasets. It represents a set of data volumes (the input data and ground truth data).

    :param input_filename: the input filename (supported by nibabel).
    :param gt_dict: a dictionary containing gt classes as keys and gt filenames as items.
    :param cache: if the data should be cached in memory or not.
    """
    
    def __init__(self, input_filename, gt_dict, cache):
        self.input_filename = input_filename
        self.gt_dict = gt_dict
        self.gt_classes = gt_dict.keys()
        self.cache = cache
        
        self.input_handle = Image(input_filename).data
        self.gt_handle = {}   
        
        for gt_class in self.gt_classes:
            self.gt_handle[gt_class] = Image(gt_dict[gt_class]).data
        
        #sanity check for dimensions
        input_shape, gt_shapes = seflf.get_shapes()
        
        for gt_shape in gt_shapes:
            if gt_shape != input_shape:
                raise RuntimeError('Input and ground truth with different dimensions.')
        
        def get_shapes(self):
            """Return the tuple (input, ground truth) representing both the input and the array of ground truth shapes."""
            
        
            

In [ ]:
class MRI2DSegmentationDataset(Dataset):
    """This is a generic class for 2D (slice-wise) segmentation datasets.
    
    :param txt_path_file: the path to a txt file containing the list of paths to input data files and gt masks.
    :param slice_axis: axis to make the slicing (default axial).
    :param cache: if the data should be cached in memory or not.
    :param transform: transformations to apply.
    """
    def __init__(self, txt_path_file, slice_axis=2, cache=True, transform=None):
        self.filenames = []
        self.read_filenames(txt_path_file)
        self.transform = transform
        self.cache = cache
        self.slice_axis = slice_axis
        self.handlers = []
        
        self._load_filenames()
    
    def _load_files():
        for input_filename, gt_dict in self.filenames:
            segset = SegmentationSet2D(input_filename, gt_dict, self.cache)
            self.handlers.append(segset)
    
    def read_filenames(txt_path_file):
        for line in open(txt_path_file, 'r'):
            if "input" in line:
                fnames=[None, {}]
                line = line.split()
                if len(line)%2:
                    raise RuntimeError('Error in filenames txt file parsing.')
                for i in range(len(line)/2):
                    try:
                        Image(line[2*i+1])
                    except Exception:
                        print "Invalid path in filenames txt file."
                    if(line[2*i]=="input"):
                        fnames[0]=line[2*i+1]
                    else:
                        fnames[1][line[2*i]]=line[2*i+1]
                self.filenames.append((fnames[0], fnames[1]))
        
                

In [1]:
print {}.keys()

[]
